In [1]:
import gc
gc.enable()

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import h2o
from h2o.automl import H2OAutoML

SEED = 2024

In [2]:
h2o.init(max_mem_size="20G")
h2o.display.toggle_user_tips('off')

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.22" 2024-01-16; OpenJDK Runtime Environment (build 11.0.22+7-post-Ubuntu-0ubuntu220.04.1); OpenJDK 64-Bit Server VM (build 11.0.22+7-post-Ubuntu-0ubuntu220.04.1, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpebfxvaxg
  JVM stdout: /tmp/tmpebfxvaxg/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpebfxvaxg/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,1 month and 18 days
H2O_cluster_name:,H2O_from_python_unknownUser_udpxy8
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,20 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [3]:
DATA_DIR = '/kaggle/input/playground-series-s4e5'

train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')

In [4]:
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

In [5]:
TARGET = 'FloodProbability'
features = [f for f in test.columns]

In [6]:
train['sum_all'] = train[features].sum(axis=1)
test['sum_all'] = test[features].sum(axis=1)

features_ext1 = list(test.columns)

In [7]:
time_limit = 60 * 60 * 8
aml = H2OAutoML(
    nfolds=10,
    max_runtime_secs=time_limit,
    stopping_metric='deviance',
    stopping_rounds=200,
    stopping_tolerance=1e-3,
    sort_metric='deviance',
    exploitation_ratio=0.1,
    seed=SEED)

aml.train(x=features_ext1, y=TARGET, training_frame=h2o.H2OFrame(train))

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_2_AutoML_1_20240501_154750


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    1058               1058                        237535                 0            7            0.827032      1             128           12.8667

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 0.0003414576515993639
RMSE: 0.018478572769544836
MAE: 0.014424332729460296
RMSLE: 0.012346913589040213
Mean Residual Deviance: 0.0003414576515993639

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 0.0003473339210224277
RMSE: 0.0186368967648165
MAE: 0.014535656159506737
RMSLE: 0.012451852039195672
Mean Residual Deviance: 0.0003473339210224277

Cross-Validation Metrics Summary: 
                        mean         sd           cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid    cv_6_valid    cv_7_valid    cv_8_valid    cv_9_valid    cv_10_valid
----------------------  -----------  -----------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  -------------
aic                     nan          0            nan           nan           nan           nan           nan           nan           nan           nan           nan           nan
loglikelihood           nan          0            nan           nan           nan           nan           nan           nan           nan           nan           nan           nan
mae                     0.0145357    3.22463e-05  0.0145222     0.0145059     0.0145249     0.0145519     0.0145711     0.0145154     0.0145684     0.0144941     0.0145128     0.0145897
mean_residual_deviance  0.000347334  1.69902e-06  0.000346873   0.000346644   0.000348393   0.000347332   0.000349366   0.000346238   0.000348203   0.000344951   0.000345201   0.000350137
mse                     0.000347334  1.69902e-06  0.000346873   0.000346644   0.000348393   0.000347332   0.000349366   0.000346238   0.000348203   0.000344951   0.000345201   0.000350137
r2                      0.866592     0.00106822   0.867823      0.866827      0.866518      0.866619      0.866042      0.866672      0.865616      0.867277      0.868103      0.864429
residual_deviance       0.000347334  1.69902e-06  0.000346873   0.000346644   0.000348393   0.000347332   0.000349366   0.000346238   0.000348203   0.000344951   0.000345201   0.000350137
rmse                    0.0186368    4.55733e-05  0.0186245     0.0186184     0.0186653     0.0186368     0.0186913     0.0186075     0.0186602     0.0185729     0.0185796     0.0187119
rmsle                   0.0124518    2.99712e-05  0.0124507     0.0124387     0.0124682     0.0124521     0.0124845     0.0124288     0.0124691     0.0124063     0.0124178     0.0125019

Scoring History: 
     timestamp            duration           number_of_trees    training_rmse         training_mae          training_deviance
---  -------------------  -----------------  -----------------  --------------------  --------------------  ----------------------
     2024-05-01 21:10:40  37 min 42.490 sec  0.0                0.05102607261300713   0.040932065962721136  0.002603660086307876
     2024-05-01 21:10:43  37 min 45.278 sec  5.0                0.0337258531969235    0.027794914211485562  0.0011374331738604351
     2024-05-01 21:10:46  37 min 47.764 sec  10.0               0.02713556802038191   0.02254204170644283   0.0007363390517887735
     2024-05-01 21:10:48  37 min 50.133 sec  15.0               0.023242263490008665  0.019144954713264335  0.0005402028121389898
     2024-05-01 21:10:50  37 min 52.459 sec  20.0               0.0206953613969591

In [8]:
preds = aml.predict(h2o.H2OFrame(test[features_ext1]))

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [9]:
sub = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')
sub[TARGET] = preds.as_data_frame()
sub[TARGET] = sub[TARGET].clip(0, 1)

/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [10]:
sub.to_csv('nb08.csv', index=False)

In [11]:
!head nb08.csv

id,FloodProbability
1117957,0.5742041340210913
1117958,0.4515217711970545
1117959,0.4523413865957091
1117960,0.471347583213163
1117961,0.4726790514648551
1117962,0.5091088828516386
1117963,0.5372881957190894
1117964,0.5293940498904599
1117965,0.4725661406274535
